# **Step 1 - Load the Required packages**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model

# **Step 2 - Load the Indian hand Sign dataset**

In [3]:
train_dir = '/content/drive/MyDrive/dataset/Train'
test_dir = '/content/drive/MyDrive/dataset/Test'
train_data = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_data = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_generator = train_data.flow_from_directory(train_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')
test_generator = test_data.flow_from_directory(test_dir, target_size=(224, 224), batch_size=32, class_mode='categorical')

Found 28673 images belonging to 35 classes.
Found 14045 images belonging to 35 classes.


# **Step 3 - Load the pre-Trained ResNet50 Model**

In [4]:
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 1s 0us/step


# **Step 4 - Freeze the weight of the Pre_trained layers**

In [5]:
for layer in base_model.layers:
  layer.trainable = False

# **Step 5 - Add a new classification layer on top of the Pre - Trained Model**

In [6]:
from keras.layers import Dense, GlobalAveragePooling2D
from keras.models import Model

# add new classifier layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
predictions = Dense(train_generator.num_classes, activation='softmax')(x)

# **Step 6 - Define New Model**

In [7]:
model = Model(inputs=base_model.input, outputs=predictions)

# **Step 7 - Compile the model with a suitable optimizier and loss funcation**

In [8]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

# **Step 8 - Train the model on the train data** 

In [9]:
num_epochs = 3  # define the number of epochs to train the model #Note : but in ideal set 35 epochs

# train the model
model.fit(train_generator, epochs=num_epochs, validation_data=test_generator)

Epoch 1/3
897/897 [==============================] - 8390s 9s/step - loss: 2.7277 - accuracy: 0.3447 - val_loss: 1.6677 - val_accuracy: 0.7043
Epoch 2/3
897/897 [==============================] - 7903s 9s/step - loss: 1.0493 - accuracy: 0.8887 - val_loss: 0.5299 - val_accuracy: 0.9628
Epoch 3/3
897/897 [==============================] - 7877s 9s/step - loss: 0.4089 - accuracy: 0.9753 - val_loss: 0.2183 - val_accuracy: 0.9861


# **Step 9 - Evaluate te model on the test data**

In [10]:
test_loss, test_accuracy = model.evaluate(test_generator)
print('Test Loss:', test_loss)
print('Test Accuracy:', test_accuracy)

439/439 [==============================] - 2580s 6s/step - loss: 0.2183 - accuracy: 0.9861
Test Loss: 0.2183074802160263
Test Accuracy: 0.9861160516738892


# **Step 10 - Save the Model**

In [11]:
model.save('model.h5')